In [7]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

In [8]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [9]:

llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [10]:
expression = "551368 / 82"
expression = expression.strip()


In [11]:
print("Evaluating expression:", expression)


Evaluating expression: 551368 / 82


In [12]:
import numexpr

try:
    result = numexpr.evaluate(expression)
except ValueError as e:
    print("Error evaluating expression:", e)
    # Further debugging or input cleaning here


In [13]:
i = 0
example_query = "What is 551368 divided by 82"
llm_math.invoke(example_query)



> Entering new LLMMathChain chain...
What is 551368 divided by 82To solve the problem of dividing 551368 by 82 using Python's numexpr library, you would create and evaluate an expression as follows:

```text
551368 / 82
```

Using numexpr to evaluate this expression:

...numexpr.evaluate("551368 / 82")...

The output from running this code is:



ValueError: unknown format from LLM: To solve the problem of dividing 551368 by 82 using Python's numexpr library, you would create and evaluate an expression as follows:

```text
551368 / 82
```

Using numexpr to evaluate this expression:

...numexpr.evaluate("551368 / 82")...

The output from running this code is:

In [5]:
llm_math.invoke({"question": "What is 551368 divided by 82?"})




> Entering new LLMMathChain chain...
What is 551368 divided by 82?To solve the problem of dividing 551368 by 82 using Python's numexpr library, we can translate this into a single line mathematical expression and execute it.

```text
551368 / 82
```

...numexpr.evaluate("551368 / 82")...



ValueError: unknown format from LLM: To solve the problem of dividing 551368 by 82 using Python's numexpr library, we can translate this into a single line mathematical expression and execute it.

```text
551368 / 82
```

...numexpr.evaluate("551368 / 82")...

In [7]:
import numexpr

result = numexpr.evaluate("551368 / 82")
print(result)  # Should output: [6724]


6724.0
